In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import matplotlib.pyplot as plt
import torchtext
import spacy
import tiktoken
import pandas as pd
import numpy as np
from gensim.corpora.dictionary import Dictionary

from tqdm.notebook import tqdm

In [26]:
#GLOBALS

block_size = 256 #This is the value of T

In [27]:
import os
en = spacy.load('en_core_web_sm')
de = spacy.load('de_core_news_sm')
#spacy.load('en_core_web_sm')
#spacy.load('de_core_news_sm')

#dataset https://nlp.stanford.edu/projects/nmt/

In [30]:
def create_dataset():
    num_examples = 10000
    vocab_en = Dictionary()

    # vocab_en.add_documents([["<START>"]])
    # vocab_en.add_documents([["<END>"]])
    vocab_en.add_documents([["<PAD>"]])

    vocab_ger = Dictionary()

    vocab_ger.add_documents([["<PAD>"]])
    vocab_ger.add_documents([["<START>"]])
    vocab_ger.add_documents([["<END>"]])

    en_max = 0 
    with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
        idx_en = []
        sentences_en = []
        for i in tqdm(range(num_examples)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            doc = en(line)
            sentence = [[token.text.lower() for token in doc]]
            if len(sentence[0]) > en_max:
                en_max = len(sentence[0])
                print(en_max)
            vocab_en.add_documents(sentence)
            sentence = ["<PAD>"] + sentence[0] + ["<PAD>"]

            if len(sentence) <= block_size:
                len_pad = block_size - len(sentence)
                sentence = sentence + len_pad*"<PAD> ".split()
                assert len(sentence) == block_size, print(len(sentence))
                idx_en.append(i)
                sentences_en.append(sentence)
            else:
                sentences_en.append(np.NaN)

    print(en_max)    
    print(len(sentences_en))
    print(len(vocab_en.token2id))

    vocab_en.save(os.getcwd()+"\\vocab\\vocab_en.pkl")


    de_max = 0 
    with open(os.getcwd()+'\\data\\train_de.txt', 'r', encoding='utf8') as f:
        idx_de = []
        sentences_de = []
        for i in tqdm(range(num_examples)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            doc = de(line)
            sentence = [[token.text.lower() for token in doc]]
            if len(sentence[0]) > de_max:
                de_max = len(sentence[0])
                print(de_max)
            vocab_ger.add_documents(sentence)
            sentence = ["<START>"] + sentence[0] + ["<END>"]
            if len(sentence) <= block_size:
                len_pad = block_size - len(sentence)
                sentence = sentence + len_pad*"<PAD> ".split()
                assert len(sentence) == block_size, print(len(sentence))
                idx_de.append(i)
                sentences_de.append(sentence)
            else:
                sentences_de.append(np.NaN)
    print(de_max)  
    print(len(sentences_de))
    print(len(vocab_ger.token2id))

    vocab_ger.save(os.getcwd()+"\\vocab\\vocab_ger.pkl")

    df = pd.DataFrame()

    df['idx_inputs'] = idx_en
    df['inputs'] = sentences_en
    df['idx_targets'] = idx_de
    df['targets'] = sentences_de

    assert len(df.loc[df['idx_inputs'] == df['idx_targets']]) == len(df), print("sentences do not align")

    df = df.dropna()
    df = df.drop(columns = ["idx_inputs", "idx_targets"], axis = 1)
    df.to_csv(os.getcwd()+"\\data\\test_data.csv", index = False)

In [31]:
create = True
if create:
    create_dataset()
    vocab_en = Dictionary.load(os.getcwd()+"\\vocab\\vocab_en.pkl")
    vocab_de = Dictionary.load(os.getcwd()+"\\vocab\\vocab_ger.pkl")
else:
    vocab_en = Dictionary.load(os.getcwd()+"\\vocab\\vocab_en.pkl")
    vocab_de = Dictionary.load(os.getcwd()+"\\vocab\\vocab_ger.pkl")

  0%|          | 0/10000 [00:00<?, ?it/s]

32
34
47
49
54
57
58
67
87
91
92
97
98
100
105
117
130
130
10000
19809


  0%|          | 0/10000 [00:00<?, ?it/s]

34
43
48
63
64
71
75
86
98
100
106
135
135
10000
30543


In [20]:
s_to_i_en = vocab_en
i_to_s_en = vocab_en.token2id

s_to_i_de = vocab_de
i_to_s_de = vocab_de.token2id

In [ ]:
#TRAIN AND VAL DATASETS

df = pd.read_csv(os.getcwd()+"\\data\\data.csv")

In [ ]:
#BATCH LOADER

def get

rough work

In [23]:
test = [0,0,0]
[s_to_i_de[i] for i in test]

['<START>', '<START>', '<START>']

In [10]:
test = "hello my name is hello name is my"
test2 = "one two three four"

doc_ = en(test)

for token in doc_:
    print(token.text.lower())
sentence_ = [[token.text for token in doc_]]
#sentence = [sentence + ['one','two']]

# sentence

sentence2 = [['one', 'two', 'hello']]

hello
my
name
is
hello
name
is
my


In [11]:
vocab = Dictionary()

vocab.add_documents(sentence_)

In [15]:
vocab_en[0]

'<PAD>'

In [ ]:
vocab.doc2idx(['hello'])

In [ ]:
vocab.save(os.getcwd()+"\\vocab\\vocab.pkl")

In [ ]:
vocab_ = Dictionary.load(os.getcwd()+"\\vocab\\vocab.pkl")

vocab_[0]
vocab_.doc2idx(['hello'])

In [ ]:
df = pd.DataFrame()

In [ ]:
vocab_en = Dictionary()

# vocab_en.add_documents([["<START>"]])
# vocab_en.add_documents([["<END>"]])
vocab_en.add_documents([["<PAD>"]])

vocab_ger = Dictionary()

vocab_ger.add_documents([["<START>"]])
vocab_ger.add_documents([["<END>"]])
vocab_ger.add_documents([["<PAD>"]])

max in german text is 238
max in english text is 342

pad both to 128

15 minutes for 100000 sentences, both german and english

328514

In [ ]:
en_max = 0 
with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
    idx_en = []
    sentences_en = []
    for i in tqdm(range(400000)):
        line = f.readline()
        line = line.replace("\n", "")
        len_pad = 0
        doc = en(line)
        sentence = [[token.text.lower() for token in doc]]
        if len(sentence[0]) > en_max:
            en_max = len(sentence[0])
            print(en_max)
        vocab_en.add_documents(sentence)
        sentence = ["<PAD>"] + sentence[0] + ["<PAD>"]
        
        idx_en.append(i)
        if len(sentence) <= 256:
            len_pad = 256 - len(sentence)
            sentence = sentence + len_pad*"<PAD> ".split()
            assert len(sentence) == 256, print(len(sentence))
            sentences_en.append(sentence)
        else:
            sentences_en.append(np.NaN)

print(en_max)    
print(len(sentences_en))
print(len(vocab_en.token2id))

vocab_en.save(os.getcwd()+"\\vocab\\vocab_en.pkl")

In [ ]:
de_max = 0 
with open(os.getcwd()+'\\data\\train_de.txt', 'r', encoding='utf8') as f:
    idx_de = []
    sentences_de = []
    for i in tqdm(range(400000)):
        line = f.readline()
        line = line.replace("\n", "")
        len_pad = 0
        doc = de(line)
        sentence = [[token.text.lower() for token in doc]]
        if len(sentence[0]) > de_max:
            de_max = len(sentence[0])
            print(de_max)
        vocab_ger.add_documents(sentence)
        sentence = ["<START>"] + sentence[0] + ["<END>"]

        idx_de.append(i)
        if len(sentence) < 256:
            len_pad = 256 - len(sentence)
            sentence = sentence + len_pad*"<PAD> ".split()
            assert len(sentence) == 256, print(len(sentence))
            sentences_de.append(sentence)
        else:
            sentences_de.append(np.NaN)

print(de_max)  
print(len(sentences_de))
print(len(vocab_ger.token2id))

vocab_ger.save(os.getcwd()+"\\vocab\\vocab_ger.pkl")

In [ ]:
assert idx_en == idx_de, print("sentences do not align")

In [ ]:
df['idx_en'] = pd.Series(idx_en)
df['inputs'] = pd.Series(sentences_en)
df['idx_targets'] = pd.Series(idx_de)
df['targets'] = pd.Series(sentences_de)

In [ ]:
df.tail()

In [ ]:
df2 = df.loc[df['idx_en'] == df['idx_targets']]

In [ ]:
df2

In [ ]:
len(df)

In [ ]:
df = df.dropna()

In [ ]:
df = df.drop(columns = ["idx_en", "idx_targets"], axis = 1)

In [ ]:
df

In [ ]:
len(df)

In [ ]:
df.to_csv(os.getcwd()+"\\data\\data.csv", index = False)

In [ ]:
df['inputs'].values[0]

In [ ]:
vocab_en.doc2idx(df['inputs'].values[0])